In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

In [0]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [0]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from hyperopt import fmin, tpe, hp, STATUS_OK
from sklearn.model_selection import cross_val_score
import time

In [0]:
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/sets_de_datos/dima_train_with_features_5.csv",
                    dtype={ \
                            'antiguedad' : np.float32, 
                            'habitaciones' : np.float32, 
                            'garages' : np.float32, 
                            'banos' : np.float32,
                            'metroscubiertos' : np.float32, 
                            'metrostotales' : np.float32, 
                            'idzona' : np.float32, 
                            'lat' : np.float32, 
                            'lng' : np.float32, 
                            'gimnasio' : np.float32, 
                            'usosmultiples' : np.float32, 
                            'piscina' : np.float32, 
                            'escuelascercanas' : np.float32,
                            'centroscomercialescercanos' : np.float32, 
                            'precio' : np.float32, 
                            'servicios_cercanos' : np.float32,
                            'utilidades_extra' : np.float32, 
                            'tipodepropiedad_mean_antiguedad' : np.float32,
                            'tipodepropiedad_mean_habitaciones ' : np.float32, 
                            'tipodepropiedad_mean_garages' : np.float32,
                            'tipodepropiedad_mean_banos' : np.float32, 
                            'tipodepropiedad_mean_metroscubiertos' : np.float32,
                            'tipodepropiedad_mean_utilidades_extra' : np.float32,
                            'tipodepropiedad_mean_servicios_cercanos' : np.float32, 
                            'ciudad_mean_antiguedad' : np.float32,
                            'ciudad_mean_habitaciones' : np.float32, 
                            'ciudad_mean_garages' : np.float32, 
                            'ciudad_mean_banos': np.float32,
                            'ciudad_mean_metroscubiertos' : np.float32, 
                            'ciudad_mean_utilidades_extra' : np.float32,
                            'ciudad_mean_servicios_cercanos' : np.float32, 
                            'provincia_mean_antiguedad' : np.float32,
                            'provincia_mean_habitaciones' : np.float32, 
                            'provincia_mean_garages' : np.float32,
                            'provincia_mean_banos' : np.float32, 
                            'provincia_mean_metroscubiertos' : np.float32,
                            'provincia_mean_utilidades_extra' : np.float32, 
                            'provincia_mean_servicios_cercanos' : np.float32,
                            'ciudad_mean_lat' : np.float32, 
                            'ciudad_mean_lng' : np.float32, 
                            'provincia_mean_lat' : np.float32,
                            'provincia_mean_lng' : np.float32, 
                            'coseno_distancia_angular_al_centro_ciudad' : np.float32,
                            'coseno_distancia_angular_al_centro_provincia' : np.float32,
                            'coseno_distancia_angular_al_centro_pais' : np.float32, 
                            'año' : np.float32, 
                            'mes' : np.float32, 
                            'dia' : np.float32,
                            'metros_no_cubiertos' : np.float32, 
                            'cantidad_espacios' : np.float32, 
                            'metros_x_espacio' : np.float32,
                            'metros_x_habitaciones' : np.float32, 
                            'metros_x_garages' : np.float32, 
                            'metros_x_banos' : np.float32,
                            'metros_x_utilidades_extra' : np.float32, 
                            'ratio_metros_cubiertos' : np.float32,
                            'distancia_minima_Terreno_comercial' : np.float32,
                            'distancia_minima_Local_Comercial' : np.float32,
                            'distancia_minima_Oficina_comercial' : np.float32,
                            'distancia_minima_Local_en_centro_comercial' : np.float32,
                            'distancia_minima_Bodega_comercial' : np.float32, 
                            'banos_preciopromedio_ciudad' : np.float32,
                            'habitaciones_preciopromedio_ciudad' : np.float32, 
                            'garages_preciopromedio_ciudad' : np.float32,
                            'banos_preciopromedio_metroscubiertos' : np.float32,
                            'habitaciones_preciopromedio_metroscubiertos' : np.float32,
                            'garages_preciopromedio_metroscubiertos' : np.float32, 
                            'precio_x_m2' : np.float32,
                            'tipodepropiedad_mean_precio' : np.float32, 
                            'titulo_cantidad_palabras_importantes' : np.float32,
                            'descripcion_cantidad_palabras_importantes' : np.float32,
                            'direccion_cantidad_palabras_importantes' : np.float32,
                            'titulo_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'descripcion_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'direccion_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'titulo_longitud_media_de_palabra' : np.float32,
                            'descripcion_longitud_media_de_palabra' : np.float32,
                            'direccion_longitud_media_de_palabra' : np.float32, 
                            'titulo_cantidad_stopwords' : np.float32,
                            'descripcion_cantidad_stopwords' : np.float32, 
                            'direccion_cantidad_stopwords' : np.float32,
                            'titulo_cantidad_signos_puntacion' : np.float32,
                            'descripcion_cantidad_signos_puntacion' : np.float32,
                            'direccion_cantidad_signos_puntacion' : np.float32,
                            'direccion_cantidad_palabras_en_mayuscula' : np.float32,
                            'direccion_cantidad_titulos' : np.float32, 
                            'titulo_cantidad_palabras_top_k' : np.float32,
                            'descripcion_cantidad_palabras_top_k' : np.float32,
                            'direccion_cantidad_palabras_top_k' : np.float32,
                            'titulo_cantidad_palabras_bottom_k' : np.float32,
                            'descripcion_cantidad_palabras_bottom_k' : np.float32,
                            'direccion_cantidad_palabras_bottom_k' : np.float32,
                            'titulo_cantidad_prefijos_top_k' : np.float32, 
                            'descripcion_cantidad_prefijos_top_k' : np.float32,
                            'direccion_cantidad_prefijos_top_k' : np.float32, 
                            'titulo_cantidad_postfijos_top_k' : np.float32,
                            'descripcion_cantidad_postfijos_top_k' : np.float32,
                            'direccion_cantidad_postfijos_top_k' : np.float32, 
                            '0' : np.float32, 
                            '1' : np.float32, 
                            '2' : np.float32, 
                            '3' : np.float32, 
                            '4' : np.float32, 
                            '5' : np.float32, 
                            '6' : np.float32,
                            '7' : np.float32
                            }
                    )

In [0]:
train.set_index('id', inplace = True)

In [0]:
TARGET = 'precio'
FEATURES = ['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'servicios_cercanos', 'utilidades_extra',
       'tipodepropiedad_mean_antiguedad', 'tipodepropiedad_mean_habitaciones',
       'tipodepropiedad_mean_garages', 'tipodepropiedad_mean_banos',
       'tipodepropiedad_mean_metroscubiertos',
       'tipodepropiedad_mean_utilidades_extra',
       'tipodepropiedad_mean_servicios_cercanos', 'ciudad_mean_antiguedad',
       'ciudad_mean_habitaciones', 'ciudad_mean_garages', 'ciudad_mean_banos',
       'ciudad_mean_metroscubiertos', 'ciudad_mean_utilidades_extra',
       'ciudad_mean_servicios_cercanos', 'provincia_mean_antiguedad',
       'provincia_mean_habitaciones', 'provincia_mean_garages',
       'provincia_mean_banos', 'provincia_mean_metroscubiertos',
       'provincia_mean_utilidades_extra', 'provincia_mean_servicios_cercanos',
       'ciudad_mean_lat', 'ciudad_mean_lng', 'provincia_mean_lat',
       'provincia_mean_lng', 'coseno_distancia_angular_al_centro_ciudad',
       'coseno_distancia_angular_al_centro_provincia',
       'coseno_distancia_angular_al_centro_pais', 'año', 'mes', 'dia',
       'metros_no_cubiertos', 'cantidad_espacios', 'metros_x_espacio',
       'metros_x_habitaciones', 'metros_x_garages', 'metros_x_banos',
       'metros_x_utilidades_extra', 'ratio_metros_cubiertos', 
       'distancia_minima_Terreno_comercial',
       'distancia_minima_Local_Comercial',
       'distancia_minima_Oficina_comercial',
       'distancia_minima_Local_en_centro_comercial',
       'distancia_minima_Bodega_comercial', 'banos_preciopromedio_ciudad',
       'habitaciones_preciopromedio_ciudad', 'garages_preciopromedio_ciudad',
       'banos_preciopromedio_metroscubiertos',
       'habitaciones_preciopromedio_metroscubiertos',
       'garages_preciopromedio_metroscubiertos', 'tipodepropiedad_mean_precio',
       'precio_x_m2', 'titulo_cantidad_palabras_importantes',
       'descripcion_cantidad_palabras_importantes',
       'direccion_cantidad_palabras_importantes',
       'titulo_cantidad_caracteres_en_palabras_importantes',
       'descripcion_cantidad_caracteres_en_palabras_importantes',
       'direccion_cantidad_caracteres_en_palabras_importantes',
       'titulo_longitud_media_de_palabra',
       'descripcion_longitud_media_de_palabra',
       'direccion_longitud_media_de_palabra', 'titulo_cantidad_stopwords',
       'descripcion_cantidad_stopwords', 'direccion_cantidad_stopwords',
       'titulo_cantidad_signos_puntacion',
       'descripcion_cantidad_signos_puntacion',
       'direccion_cantidad_signos_puntacion',
       'direccion_cantidad_palabras_en_mayuscula',
       'direccion_cantidad_titulos', 'titulo_cantidad_palabras_top_k',
       'descripcion_cantidad_palabras_top_k',
       'direccion_cantidad_palabras_top_k',
       'titulo_cantidad_palabras_bottom_k',
       'descripcion_cantidad_palabras_bottom_k',
       'direccion_cantidad_palabras_bottom_k',
       'titulo_cantidad_prefijos_top_k', 'descripcion_cantidad_prefijos_top_k',
       'direccion_cantidad_prefijos_top_k', 'titulo_cantidad_postfijos_top_k',
       'descripcion_cantidad_postfijos_top_k',
       'direccion_cantidad_postfijos_top_k', '0', '1', '2', '3', '4', '5', '6',
       '7']

In [0]:
drop_columns_1 = ['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 
                  'ciudad', 'provincia', 'fecha', 'mes', 'dia',
                  'direccion_cantidad_palabras_bottom_k',
                  'descripcion_cantidad_palabras_bottom_k',
                  'titulo_cantidad_palabras_bottom_k', '7', '0', 'usosmultiples', '2',
                  'escuelascercanas', 'centroscomercialescercanos',
                  'direccion_cantidad_stopwords', 'direccion_cantidad_signos_puntacion',
                  'servicios_cercanos', '4', 'direccion_cantidad_palabras_top_k',
                  'direccion_cantidad_palabras_en_mayuscula',
                  'titulo_cantidad_signos_puntacion', 'piscina',
                  'direccion_cantidad_titulos', 'titulo_cantidad_palabras_top_k',
                  'titulo_cantidad_stopwords', '3', 'direccion_cantidad_prefijos_top_k',
                  'titulo_cantidad_prefijos_top_k', 'gimnasio',
                  'titulo_cantidad_postfijos_top_k', 'direccion_cantidad_postfijos_top_k',
                  'metros_no_cubiertos', 'titulo_cantidad_palabras_importantes',
                  'ratio_metros_cubiertos', 'utilidades_extra', 'provincia_mean_lng',
                  'direccion_cantidad_palabras_importantes', '1',
                  'provincia_mean_utilidades_extra', 'provincia_mean_garages', '6',
                  'provincia_mean_lat', 'provincia_mean_banos', 'habitaciones',
                  'ciudad_mean_lng', 'ciudad_mean_garages',
                  'provincia_mean_metroscubiertos',
                  'descripcion_cantidad_signos_puntacion',
                  'descripcion_cantidad_prefijos_top_k', 'garages',
                  'direccion_longitud_media_de_palabra',
                  'descripcion_cantidad_postfijos_top_k',
                  'descripcion_cantidad_palabras_importantes', '5',
                  'tipodepropiedad_mean_servicios_cercanos',
                  'titulo_cantidad_caracteres_en_palabras_importantes',
                  'ciudad_mean_servicios_cercanos', 'titulo_longitud_media_de_palabra',
                  'descripcion_cantidad_palabras_top_k', 'descripcion_cantidad_stopwords',
                  'provincia_mean_servicios_cercanos',
                  'descripcion_cantidad_caracteres_en_palabras_importantes',
                  'ciudad_mean_banos', 'distancia_minima_Local_Comercial',
                  'provincia_mean_habitaciones', 'descripcion_longitud_media_de_palabra',
                  'direccion_cantidad_caracteres_en_palabras_importantes',
                  'distancia_minima_Terreno_comercial', 'ciudad_mean_lat',
                  'ciudad_mean_metroscubiertos', 'metros_x_utilidades_extra',
                  'coseno_distancia_angular_al_centro_provincia',
                  'tipodepropiedad_mean_banos'
                  ]
FEATURES_1 = list(set(FEATURES) - set(drop_columns_1))

In [0]:
X = train[FEATURES_1]
y = train[TARGET]

In [0]:
K = 5
def objective(hyper_parametros):

    rf_regressor = RandomForestRegressor(**hyper_parametros)
    
    score = cross_val_score(rf_regressor, X, y, 
                            scoring='neg_mean_absolute_error', cv=K).mean()
    
    print("SCORE: {:.3f} params {}".format(score, hyper_parametros))
    
    return {'loss': -score, 'status': STATUS_OK}

In [0]:
space = {
    'max_depth': hp.randint("max_depth", 10) + 10,  
    'max_features': hp.randint("max_features", 10) + 10, 
    'n_estimators': hp.randint("n_estimators", 300) + 200,
    'min_samples_leaf': hp.randint('min_samples_leaf', 10) + 1,
    'min_samples_split' : hp.randint('min_samples_split', 10) + 2,
}

In [33]:
N_ITER = 10
t0 = time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=N_ITER)
t1 = time.time()
print('Tiempo = {0:.2f} horas'.format((t1 - t0)/3600))

SCORE: -619363.478 params {'max_depth': 12, 'max_features': 19, 'min_samples_leaf': 5, 'min_samples_split': 8, 'n_estimators': 401}
SCORE: -608004.989 params {'max_depth': 13, 'max_features': 15, 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 331}
SCORE: -569135.238 params {'max_depth': 19, 'max_features': 13, 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 259}
SCORE: -586984.764 params {'max_depth': 17, 'max_features': 12, 'min_samples_leaf': 9, 'min_samples_split': 8, 'n_estimators': 290}
SCORE: -601835.409 params {'max_depth': 14, 'max_features': 13, 'min_samples_leaf': 9, 'min_samples_split': 3, 'n_estimators': 216}
SCORE: -570445.106 params {'max_depth': 19, 'max_features': 15, 'min_samples_leaf': 4, 'min_samples_split': 11, 'n_estimators': 387}
SCORE: -575482.088 params {'max_depth': 18, 'max_features': 15, 'min_samples_leaf': 1, 'min_samples_split': 11, 'n_estimators': 432}
SCORE: -636697.826 params {'max_depth': 11, 'max_features': 13, 'min_sampl

SCORE: -619363.478 params {'max_depth': 12, 'max_features': 19, 'min_samples_leaf': 5, 'min_samples_split': 8, 'n_estimators': 401}  
SCORE: -608004.989 params {'max_depth': 13, 'max_features': 15, 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 331}  
SCORE: -569135.238 params {'max_depth': 19, 'max_features': 13, 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 259}  
SCORE: -586984.764 params {'max_depth': 17, 'max_features': 12, 'min_samples_leaf': 9, 'min_samples_split': 8, 'n_estimators': 290}  
SCORE: -601835.409 params {'max_depth': 14, 'max_features': 13, 'min_samples_leaf': 9, 'min_samples_split': 3, 'n_estimators': 216}  
SCORE: -570445.106 params {'max_depth': 19, 'max_features': 15, 'min_samples_leaf': 4, 'min_samples_split': 11, 'n_estimators': 387}  
SCORE: -575482.088 params {'max_depth': 18, 'max_features': 15, 'min_samples_leaf': 1, 'min_samples_split': 11, 'n_estimators': 432}  
SCORE: -636697.826 params {'max_depth': 11, 'max_features': 13, 'min_samples_leaf': 8, 'min_samples_split': 7, 'n_estimators': 467}  
SCORE: -605773.857 params {'max_depth': 14, 'max_features': 10, 'min_samples_leaf': 10, 'min_samples_split': 4, 'n_estimators': 479}  
SCORE: -638096.250 params {'max_depth': 11, 'max_features': 12, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 261}  

SCORE: -718602.438 params {'max_depth': 8, 'max_features': 13, 'min_samples_leaf': 3, 'min_samples_split': 7, 'n_estimators': 423}  
SCORE: -827641.060 params {'max_depth': 5, 'max_features': 14, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 389}  
SCORE: -722483.887 params {'max_depth': 8, 'max_features': 10, 'min_samples_leaf': 10, 'min_samples_split': 10, 'n_estimators': 299}  
SCORE: -738378.851 params {'max_depth': 8, 'max_features': 5, 'min_samples_leaf': 2, 'min_samples_split': 8, 'n_estimators': 302}  
SCORE: -703435.084 params {'max_depth': 9, 'max_features': 7, 'min_samples_leaf': 5, 'min_samples_split': 4, 'n_estimators': 232}  
SCORE: -703731.643 params {'max_depth': 9, 'max_features': 7, 'min_samples_leaf': 10, 'min_samples_split': 7, 'n_estimators': 250}  
SCORE: -639390.435 params {'max_depth': 12, 'max_features': 8, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 520}  
SCORE: -763658.722 params {'max_depth': 7, 'max_features': 6, 'min_samples_leaf': 7, 'min_samples_split': 2, 'n_estimators': 354}  
SCORE: -584129.352 params {'max_depth': 17, 'max_features': 13, 'min_samples_leaf': 8, 'min_samples_split': 5, 'n_estimators': 456}  
SCORE: -690877.390 params {'max_depth': 9, 'max_features': 14, 'min_samples_leaf': 8, 'min_samples_split': 9, 'n_estimators': 332}  
SCORE: -588346.328 params {'max_depth': 17, 'max_features': 10, 'min_samples_leaf': 8, 'min_samples_split': 3, 'n_estimators': 214}  
SCORE: -594802.270 params {'max_depth': 15, 'max_features': 12, 'min_samples_leaf': 6, 'min_samples_split': 10, 'n_estimators': 480}  
SCORE: -790487.664 params {'max_depth': 6, 'max_features': 9, 'min_samples_leaf': 7, 'min_samples_split': 6, 'n_estimators': 496}  
SCORE: -748579.612 params {'max_depth': 7, 'max_features': 14, 'min_samples_leaf': 6, 'min_samples_split': 5, 'n_estimators': 352}  
SCORE: -622046.260 params {'max_depth': 13, 'max_features': 9, 'min_samples_leaf': 6, 'min_samples_split': 6, 'n_estimators': 300}  
SCORE: -828315.773 params {'max_depth': 5, 'max_features': 13, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 207}  
SCORE: -827619.816 params {'max_depth': 5, 'max_features': 14, 'min_samples_leaf': 9, 'min_samples_split': 11, 'n_estimators': 583}  